In [1]:
using Pkg
Pkg.add(["Images", "ImageIO", "FileIO", "Random", "ProgressMeter", "Plots", "Colors", "Statistics"])

    Updating registry at `~/.julia/registries/General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Resolving package versions...
   Installed JpegTurbo_jll ──────────────────── v3.1.1+0
   Installed Libmount_jll ───────────────────── v2.41.0+0
   Installed ImageSegmentation ──────────────── v1.8.1
   Installed ImageIO ────────────────────────── v0.6.8
   Installed TiledIteration ─────────────────── v0.5.0
   Installed AxisArrays ─────────────────────── v0.4.7
   Installed OffsetArrays ───────────────────── v1.16.0
   Installed TiffImages ─────────────────────── v0.10.2
   Installed LERC_jll ───────────────────────── v4.0.1+0
   Installed GR_jll ─────────────────────────── v0.73.13+0
   Installed FFTW ───────────────────────────── v1.8.1
   Installed HistogramThresholding ──────────── v0.3.1
   Installed PNGFiles ───────────────────────── v0.4.4
   Installed JpegTurbo ──────────────────────── v0.1.6
   Installed ImageMagick ────────────────────── v1.2.1
   

In [ ]:
using Images, ImageIO, FileIO
using Random, ProgressMeter
using Plots, Colors, Statistics

# Параметры генетического алгоритма
const POPULATION_SIZE = 50
const MUTATION_RATE = 0.1
const MAX_GENERATIONS = 1000
const EPSILON_BOOLEAN = 0.05  # 5% несовпадений
const EPSILON_EUCLIDEAN = 0.1  # 10% от максимального расстояния

"""
Создание эталонного вектора цветов
"""
function create_reference_vector(size::Int)
    return rand(0:255, size*size, 3)
end

"""
Инициализация популяции случайными векторами
"""
function initialize_population(vector_size::Int, pop_size::Int)
    return [rand(0:255, vector_size, 3) for _ in 1:pop_size]
end

"""
Булево расстояние - доля несовпадающих элементов
"""
function boolean_distance(vector1::Array{Int,2}, vector2::Array{Int,2})
    return mean(any(vector1 .!= vector2, dims=2))
end

"""
Евклидово расстояние в десятичной системе
"""
function euclidean_distance(vector1::Array{Int,2}, vector2::Array{Int,2})
    return sqrt(sum((vector1 - vector2).^2)) / (length(vector1) * 255)
end

"""
Выбор родителей методом рулетки
"""
function select_parents(population, fitness, num_parents)
    fitness_inv = 1.0 ./ (fitness .+ 1e-10)  # Инвертируем, т.к. меньше расстояние - лучше
    probs = fitness_inv ./ sum(fitness_inv)
    indices = sample(1:length(population), Weights(probs), num_parents, replace=true)
    return [population[i] for i in indices]
end

"""
Кроссинговер на уровне генов (каждое значение - ген)
"""
function crossover_gene_level(parent1::Array{Int,2}, parent2::Array{Int,2})
    child = copy(parent1)
    mask = rand(size(parent1)...) .< 0.5
    child[mask] = parent2[mask]
    return child
end

"""
Кроссинговер на уровне вектора (весь вектор - ген)
"""
function crossover_vector_level(parent1::Array{Int,2}, parent2::Array{Int,2})
    if rand() < 0.5
        return copy(parent1)
    else
        return copy(parent2)
    end
end

"""
Мутация - случайное изменение значений
"""
function mutate!(vector::Array{Int,2}, rate::Float64)
    mask = rand(size(vector)...) .< rate
    vector[mask] = rand(0:255, count(mask))
    return vector
end

"""
Преобразование вектора в изображение
"""
function vector_to_image(vector::Array{Int,2}, size::Int)
    img = reshape(vector, size, size, 3)
    img = permutedims(img, (2, 1, 3))  # Переставляем оси для правильного отображения
    return RGB.(img[:,:,1] ./ 255, img[:,:,2] ./ 255, img[:,:,3] ./ 255)
end

"""
Запуск генетического алгоритма
"""
function run_genetic_algorithm(size::Int, distance_func::Function, crossover_func::Function, epsilon::Float64)
    reference = create_reference_vector(size)
    vector_size = size * size
    population = initialize_population(vector_size, POPULATION_SIZE)
    
    images = []
    best_fitness_history = Float64[]
    
    p = Progress(MAX_GENERATIONS, desc="Генерации: ")
    for generation in 1:MAX_GENERATIONS
        # Оценка приспособленности
        fitness = [distance_func(ind, reference) for ind in population]
        best_idx = argmin(fitness)
        best_fitness = fitness[best_idx]
        push!(best_fitness_history, best_fitness)
        
        # Сохранение текущего состояния
        if generation % 10 == 0 || generation == MAX_GENERATIONS
            best_image = vector_to_image(population[best_idx], size)
            ref_image = vector_to_image(reference, size)
            
            p1 = plot(ref_image, title="Эталон", axis=nothing, size=(300, 300))
            p2 = plot(best_image, title="Поколение $generation\nРасстояние: $(round(best_fitness, digits=4))", 
                      axis=nothing, size=(300, 300))
            
            p_combined = plot(p1, p2, layout=(1, 2), size=(600, 300))
            push!(images, p_combined)
        end
        
        # Проверка критерия остановки
        if best_fitness < epsilon
            break
        end
        
        # Выбор родителей и создание нового поколения
        new_population = [population[best_idx]]  # Элитизм
        
        while length(new_population) < POPULATION_SIZE
            parents = select_parents(population, fitness, 2)
            child = crossover_func(parents[1], parents[2])
            mutate!(child, MUTATION_RATE)
            push!(new_population, child)
        end
        
        population = new_population
        next!(p)
    end
    
    return images, best_fitness_history, length(best_fitness_history)
end

"""
Сохранение GIF-анимации
"""
function save_gif(images, filename; fps=5)
    anim = @animate for img in images
        plot(img)
    end
    
    gif(anim, filename, fps=fps)
end

"""
Запуск эксперимента с заданными параметрами
"""
function run_experiment(size::Int, distance_name::String, crossover_name::String)
    println("Запуск эксперимента: размер $(size)x$(size), расстояние $distance_name, кроссинговер $crossover_name")
    
    if distance_name == "boolean"
        distance_func = boolean_distance
        epsilon = EPSILON_BOOLEAN
    else
        distance_func = euclidean_distance
        epsilon = EPSILON_EUCLIDEAN
    end
    
    if crossover_name == "gene"
        crossover_func = crossover_gene_level
    else
        crossover_func = crossover_vector_level
    end
    
    images, fitness_history, generations = run_genetic_algorithm(size, distance_func, crossover_func, epsilon)
    
    # Сохранение GIF
    filename = "convergence_$(size)x$(size)_$(distance_name)_$(crossover_name).gif"
    save_gif(images, filename)
    
    return filename, fitness_history, generations
end

# Запуск всех экспериментов
function run_all_experiments()
    sizes = [3, 5, 9, 21]
    distances = [("boolean", "Булево"), ("euclidean", "Евклидово")]
    crossovers = [("gene", "Ген-уровень"), ("vector", "Вектор-уровень")]

    results = Dict()

    for size in sizes
        for (dist_code, dist_name) in distances
            for (cross_code, cross_name) in crossovers
                key = "$(size)x$(size)_$(dist_code)_$(cross_code)"
                gif_file, fitness_history, generations = run_experiment(size, dist_code, cross_code)
                results[key] = Dict(
                    "gif" => gif_file,
                    "fitness" => fitness_history,
                    "generations" => generations,
                    "size" => size,
                    "distance" => dist_name,
                    "crossover" => cross_name
                )
            end
        end
    end

    # Вывод результатов
    for (key, data) in results
        println("Эксперимент: $(data["size"])x$(data["size"]), $(data["distance"]) расстояние, $(data["crossover"]) кроссинговер")
        println("Поколений до сходимости: $(data["generations"])")
        println("Итоговое расстояние: $(round(data["fitness"][end], digits=6))")
        println("GIF-файл: $(data["gif"])")
        println("-" ^ 50)
    end
    
    return results
end

# Запуск всех экспериментов
Random.seed!(42)  # Для воспроизводимости результатов
results = run_all_experiments()